In [ ]:
import torch
import config
import numpy as np
import matplotlib.pyplot as plt
import os
from Alg.solving_algorithm import ModelGenerator
from CustomModels.my_models import Integrator
from CustomModels.my_models import weighted_amount,renormolize_distribution
from aml.plotting import *
from Losses.Losses import *
from sklearn import decomposition
from tqdm import tqdm
from scipy.spatial import ConvexHull

def adjust_alpha(alpha_n):
    for j in range(len(alpha_n)):
        if alpha_n[j]<0.0:
            alpha_n[j] = 0.0
        elif alpha_n[j] > 1.0:
            alpha_n[j] = 1.0
    alpha_n = alpha_n/np.sum(alpha_n)
    return alpha_n 

def Temp(k,T_0,max_iter):
    # return T_0/np.log(k+2)
    # return T_0/k
    a = -T_0/max_iter
    b = T_0
    return a*k+b

def ProbOfChangeToNewParams(L_new, L_old, beta, current_temp):
    H_ = 0.0
    if L_new < L_old:
        H_ = 1.0
    else:
        H_=  np.minimum(1.0,np.exp((L_old-L_new)/(beta*current_temp)))
        # H_ = 0.0
    return H_

def UpdateMu(mu_current,mu_new,epsilon,ProbOfUpdate):
    if ProbOfUpdate == 1.0:
        # new_ = epsilon*mu_current + (1-epsilon)*mu_new
        new_ = mu_new
        new_ = new_/np.sum(new_)
        return  new_, True
    if ProbOfUpdate == 0.0:
        return  mu_current, False
    
    v_ = ProbOfUpdate*1000
    coin_=  np.random.randint(0,1001)
    if coin_ <= v_:
        new_ = mu_new
        # new_ = epsilon*mu_current + (1-epsilon)*mu_new
        new_ = new_/np.sum(new_)
        return new_, True
    else:
        return mu_current,False
        
def GenAlphaVec(mu_current, value_of_noise):
    noise = np.random.rand(len(mu_current))
    noise = noise/np.sum(noise)
    new_ = (1.0-value_of_noise)*mu_current + value_of_noise* noise
    new_ = new_/np.sum(new_)
    return new_

def adjust_alpha(alpha_n):
    for j in range(len(alpha_n)):
        if alpha_n[j]<0.0:
            alpha_n[j] = 0.001
        elif alpha_n[j] > 1.0:
            alpha_n[j] = 1.0
    alpha_n = alpha_n/np.sum(alpha_n)
    return alpha_n 

def grad_descent_from_alpha(alpha_vec,p_0, shared_integration_supports,a,board,exp_meta,
                            MAX_ITER,MAX_GENERATIONS,T_0):
    d= len(alpha_vec)   
    alpha_n = np.copy(alpha_vec)
    alpha_n = alpha_n/np.sum(alpha_n)
    p_n = renormolize_distribution(p_0,a,alpha_n)
    L_n = get_L2_Distrib4D(p_n, shared_integration_supports)

    for ITER in range(MAX_ITER):
        board.Push(experiment_metadata=exp_meta,
        x=ITER,y= L_n, label='L')
        if ITER % 10 ==0:
            print('ITER {} L_n {}'.format(ITER,L_n))
            # print(alpha_n)
        else:
            print('ITER {} L_n {}'.format(ITER,L_n))

        p_n=  renormolize_distribution(p_n,a,alpha_n)
        L_n = get_L2_Distrib4D(p_n,shared_integration_supports)
        gradient_ = np.zeros(shape=(d,))
        epsilon_ = 10**(-1)
        for j in range(d):
            if alpha_n[j] < epsilon_:
                alpha_1 = np.copy(alpha_n)
                alpha_1[j] = alpha_1[j] + epsilon_
                p_1 = renormolize_distribution(p_n,a,alpha_1)
                L_1 = get_L2_Distrib4D(p_1,shared_integration_supports)
                gradient_[j] = (L_1 - L_n)/epsilon_
                continue
            if alpha_n[j] > 1.0-epsilon_:
                alpha_2 = np.copy(alpha_n)
                alpha_2[j] = alpha_2[j] - epsilon_
                p_2 = renormolize_distribution(p_n,a,alpha_2)
                L_2 = get_L2_Distrib4D(p_2,shared_integration_supports)
                gradient_[j] = (L_n-L_2)/epsilon_
                continue

            alpha_1 = np.copy(alpha_n)
            alpha_1[j] = alpha_1[j] + epsilon_
            alpha_2 = np.copy(alpha_n)
            alpha_2[j] = alpha_2[j] - epsilon_

            p_1 = renormolize_distribution(p_n,a,alpha_1)
            p_2 = renormolize_distribution(p_n,a,alpha_2)

            L_1 = get_L2_Distrib4D(p_1,shared_integration_supports)
            L_2 = get_L2_Distrib4D(p_2,shared_integration_supports)
            gradient_[j] = (L_1 - L_2)/(2*epsilon_)
        board.Push(experiment_metadata=exp_meta,
        x=ITER,y= np.linalg.norm(gradient_), label='grad norm')
        lambda_vec = np.logspace(start=4,stop=-4,num=10)
        lambda_best = None
        loss_current = L_n
        ls = []
        lambda_ls = []
        for lambda_ in lambda_vec:
            alpha_copy = np.copy(alpha_n)
            alpha_after = alpha_copy - lambda_*gradient_
            if np.sum(alpha_after < 0.0)==d:
                continue
            alpha_after = adjust_alpha(alpha_after)        
            p_after = renormolize_distribution(p_n,a,alpha_after)
            L_after = get_L2_Distrib4D(p_after,shared_integration_supports)
            ls.append(L_after)
            lambda_ls.append(lambda_)
            if L_after<loss_current:
                loss_current = L_after
                lambda_best = lambda_
        arg_best = np.argsort(ls)[0]
        left_pos = np.maximum(0, arg_best-1)
        right_pos = np.minimum(len(ls)-1, arg_best+1)

        left = lambda_ls[left_pos]
        right = lambda_ls[right_pos]

        lambda_vec = np.linspace(left,right,10)
        addls = []
        addlambda_ls = []
        for lambda_ in lambda_vec:
            alpha_copy = np.copy(alpha_n)
            alpha_after = alpha_copy - lambda_*gradient_
            if np.sum(alpha_after < 0.0)==d:
                continue
            alpha_after = adjust_alpha(alpha_after)        
            p_after = renormolize_distribution(p_n,a,alpha_after)
            L_after = get_L2_Distrib4D(p_after,shared_integration_supports)
            addls.append(L_after)
            addlambda_ls.append(lambda_)
            if L_after<loss_current:
                loss_current = L_after
                lambda_best = lambda_
        if lambda_best == None:
            alpha_n = alpha_n + epsilon_*np.random.rand(d)
            alpha_n = alpha_n/np.sum(alpha_n)
        else:
            fig,ax = plt.subplots()
            ax.plot(lambda_ls, ls)
            ax.set_xscale('log')
            plt.show()

            alpha_n = alpha_n - lambda_best*gradient_
            alpha_n = adjust_alpha(alpha_n)
        print(alpha_n)

    p_n = renormolize_distribution(p_n,a,alpha_n)
    return p_n, L_n,alpha_n

    # alpha_best_ = None
    # T_n = T_0
    # p_n = p_0
    # L_best = 10**99
    # L_current = 10**99
    # # all_losses = []
    # # while True:
    # #     p_n=  renormolize_distribution(p_n,a,alpha_n)
    # #     L_n = get_L2_Distrib4D(p_n,shared_integration_supports)
        
    # #     n +=1    
    # l_all = []
    # for ITER in tqdm(range(MAX_ITER)):
    #     sigma_n = T_n/T_0
    #     for j in range(MAX_GENERATIONS):
    #         alpha_ = GenAlphaVec(mu_current=alpha_n, value_of_noise=0.3*sigma_n)
    #         p_ = renormolize_distribution(p_n, a, alpha_)
    #         L_new = get_L2_Distrib4D(p_,shared_integration_supports)
    #         H_ = ProbOfChangeToNewParams(L_new,L_current, 1.0,T_n)
    #         board.Push(experiment_metadata=exp_meta,
    #             x=ITER,y= H_, label='H')
    #         alpha_updated, is_update = UpdateMu(alpha_n,alpha_,epsilon=0.8,ProbOfUpdate=H_)
    #         if is_update:
    #             board.Push(experiment_metadata=exp_meta,
    #             x=ITER,y= L_new, label='L')

    #         if is_update and (L_new<L_current):
    #             L_current = L_new
    #             alpha_n = alpha_updated
    #             break
    #         elif is_update and not (L_new<L_current):
    #             L_current = L_new
    #             alpha_n = alpha_updated

    #         if L_new < L_best:
    #             L_best = L_new
    #             alpha_best_ = np.copy(alpha_updated)
    #     if ITER % 10 ==0:
    #         print(alpha_n)
    #     T_n = Temp(ITER,T_0,MAX_ITER)
    #     board.Push(experiment_metadata=exp_meta,
    #             x=ITER,y= T_n, label='T')
    # p_n = renormolize_distribution(p_n,a,alpha_n)
    # return p_n, L_current,alpha_n


def approx_equal(x1,x2,precision):
    if np.absolute(x1-x2) < precision:
        return True
    else:
        return False

In [ ]:
board = TensorBoard(tensorboard_exe_path=config.tensorboard_path,
                    logdir=os.path.join(config.task_dir, 'rules_descent_log'),
                    port= '64001')

In [ ]:
mg = ModelGenerator(rules=config.rules,
                            cache_dir=config.Phi_cache_dir,
                            clear_cache=False)
shared_integration_supports = Integrator(dir_=config.integrator_dir,
                                        shared_data=mg.shared_data,
                                        clear_cache=True).shared_integration_supports
N=1000
all_p = [torch.load(os.path.join(mg.cache_dir, 'distrib4D_{}.txt'.format(i))) for i in range(N)]
all_v=  torch.load(os.path.join(config.task_dir, 'L2_for_Phi.txt'))
nr = len(all_p[0].z_list)
uniform_distrib_of_rules= np.ones(shape=(nr,))/nr
a, h, f, coeff_list = mg.shared_data['ahfcoeff_list']
a = [a[el].detach().numpy() for el in range(len(a))]
for i in range(N):
    all_p[i] = renormolize_distribution(all_p[i],a,uniform_distrib_of_rules)

choose start distribtuion by random choise

In [ ]:
# p_0 = all_p[np.argmin(all_v)]
p_0 = all_p[np.random.randint(0,N)]


make gradiend descent on distribution in $R^{n_{r}}$

In [ ]:
exp_metadata = 'gradien_descent'+get_time()
board.InitExperiment(experiment_metadata= exp_metadata)

In [ ]:
alpha_n = np.copy(uniform_distrib_of_rules)
T_0 = 0.002
MAX_GENERATIONS = 1000
MAX_ITER = 300

In [ ]:
p_best, l_best,alpha_n = grad_descent_from_alpha(alpha_n, p_0, shared_integration_supports,a,board,exp_metadata,MAX_ITER,MAX_GENERATIONS,T_0)